# YOLOv5 Weak-to-Strong Training

## Setup

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="PZvJbpNUYuJJDMh8nypC")
project = rf.workspace("gdit").project("aerial-airport")
dataset = project.version(1).download("yolov5")

In [ ]:
epochs = 50
batch_size = 16
img_size = 416
weak_model = "yolov5n"
strong_model = "yolov5l"

In [ ]:
# full_dataset_path = dataset.location
full_dataset_path = "/content/drive/MyDrive/Colab\ Notebooks/weak-to-strong-generalization/Cars-Detection"

## Train strong ceiling model

In [ ]:
# Make sure that [full_dataset_path] contains the original labels
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {full_dataset_path}/data.yaml --weights {strong_model}.pt --cache

In [ ]:
from google.colab import files
files.download('./runs/train/exp')

In [ ]:
%cp -r "./runs/train/exp" "/content/drive/MyDrive/Colab Notebooks/weak-to-strong-generalization/yolov5l-ceiling-Cars-Detection"

## Train weak model

In [ ]:
reduced_dataset_path = '/content/drive/MyDrive/Colab\ Notebooks/weak-to-strong-generalization/Cars-Detection-reduced'

In [ ]:
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {reduced_dataset_path}/data.yaml --weights {weak_model}.pt --cache

In [ ]:
%cp -r "./runs/train/exp2" "/content/drive/MyDrive/Colab Notebooks/weak-to-strong-generalization/yolov5n-weak-Cars-Detection"

## Generate weak labels

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img {img_size} --conf 0.1 --source {full_dataset_path}/train/images --save-txt

In [ ]:
#display inference on ALL test images

# import glob
# from IPython.display import Image, display

# for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
#     display(Image(filename=imageName))
#     print("\n")

In [ ]:
# Replace original labels with predicted weak labels
%rm -r {full_dataset_path}/train/labels
%cp -r runs/detect/exp/labels {full_dataset_path}/train

In [ ]:
%cp -r "./runs/detect/exp/labels" "/content/drive/MyDrive/Colab Notebooks/weak-to-strong-generalization/Cars-Detection-yolov5n-weak-labels"

## Train strong student model

In [ ]:
# Make sure that [full_dataset_path] contains the weak labels
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {full_dataset_path}/data.yaml --weights {strong_model}.pt

In [ ]:
%cp -r "./runs/train/exp3" "/content/drive/MyDrive/Colab Notebooks/weak-to-strong-generalization/yolov5l-student-yolov5n-Cars-Detection"